In [1]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.by import By
import _pickle

In [2]:
#fonction qui recupere les tags avec comme attribut data-ar-restaurant-id
def restaurants_id (tag):
    return tag.has_attr('data-ar-restaurant-id')

In [3]:
#fonction qui recupere les tags avec comme attribut data-product-id
def plats_id (tag):
    return tag.has_attr('data-product-id')

In [4]:
#Fonction qui recupere les plats d'un restaurant donné et les ajoute à une liste
def recuperer_liste_plats(link, id_restaurant, liste_plats):
    req_carte = requests.get(link)

    page_carte = req_carte.text
    soup_carte = BeautifulSoup(page_carte, 'html.parser')

    plats = soup_carte.find_all(plats_id)
    
    for plat in plats:
        soup_p = BeautifulSoup(str(plat) , 'html.parser')
        #print (soup_p.prettify())
        
        #id
        id_plat = soup_p.find("li")["data-product-id"]

        #On vérifie que le plat ajouté n'est pas deja dans la liste
        isInList = False
        for p in liste_plats:
            if p["id"]==id_plat:
                isInList = True

        if not isInList:
            #nom du plat
            name = soup_p.find("h4" , {"class" : "name"}).find("span").text

            #logo
            logo = ""
            try:
                logo = soup_p.find("img" )['src']
            except:
                pass
            
            #prix
            prix = soup_p.find("div" , {"class" : "price"}).text
            prix = " ".join(prix.split())

            #description
            description = soup_p.find("div" , {"class" : "description"}).text

            objet = {
                "id_restaurant" : id_restaurant,
                "id" : id_plat,
                "name" : name,
                "prix" : prix,
                "description" : description,
                "logo" : logo
            }
            #print(objet["prix"])

            liste_plats.append(objet)

In [5]:
#Fonction qui recupere les donnees des restaurants sur une page alloresto et les ajoute à une liste
def recuperer_liste_restaurants(codeSource, liste_restaurants, liste_plats):
    #req = requests.get(link)
    #page = req.text
    soup = BeautifulSoup(codeSource, 'html.parser')
    restaurants = soup.find_all(restaurants_id)
    i=0

    for restaurant in restaurants:
        restaurant = str(restaurant) 
        soup_r = BeautifulSoup(restaurant, 'html.parser')

        #link
        link = soup_r.find("h5" , {"class" : "name"}).find("a")["href"]

        #On vérifie que le restaurant n'est pas déjà dans la liste
        isInList = False
        for r in liste_restaurants:
            if r["link"]==link:
                isInList = True

        if not isInList:

            #id
            id_restaurant = i
            i = i + 1

            #name
            name = soup_r.find("h5" , {"class" : "name"}).text.replace("\n","")

            #logo
            logo = ""
            try:
                logo = soup_r.find("img" )['src']
            except:
                pass
            
            #address
            address = soup_r.find("div" , {"class" : "address"}).text
            address = " ".join(address.split())

            #specialties
            specialties = [x.text for x in soup_r.find("div" , {"class" : "cuisines"}).find_all("li")]

            #work time 
            #TODO: A RETRAVAILLER
            work_time = None

            #minimum_order
            temp = soup_r.find("li", {"class" : "minimum-order"})
            if temp is not None:
                minimum_order = temp.find("strong").text
                minimum_order = " ".join(minimum_order.split())
            else : 
                minimum_order = "0€"

            #print(type(name))
            objet = {
                "id" : id_restaurant,
                "name" : name,
                "logo" : logo,
                "address" : address,
                "specialties" : specialties,
                "work_time" : work_time,
                "minimum_order" : minimum_order,
                "link" : link
            }

            liste_restaurants.append(objet)
            #Appel de la fonction ajoutant la liste des plats
            recuperer_liste_plats(link, id_restaurant, liste_plats)

    #print(liste_restaurants)

In [7]:
#test = json.dumps(liste_plats)
#test

In [8]:
def recuperer_page_alloresto(link):
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # go to the page
    driver.get(link)

    inputElement = driver.find_element_by_css_selector("button.show-more")
    inputElement.click()

    sortieBoucle = False 
    #i = 0
    while sortieBoucle is False:
        try:
            # attendre que la page charge les restaurant supplémentaires
            WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'show-more')))#EC.visibility_of(inputElement))
            #print(i)
            #i = i + 1
            #On charge de nouveaux restaurants
            inputElement = driver.find_element_by_css_selector("button.show-more")
            #print(inputElement)
            inputElement.click()
        except:
            sortieBoucle = True
            #raise

    #recuperer le code source de la page
    codeSource = driver.page_source
    #On quitte le web driver
    driver.quit()
    #print ("done")
    return codeSource

test = BeautifulSoup(codeSource, 'html.parser')
print(test.prettify())

In [9]:
liste_restaurants = []
liste_plats = []
link="https://www.alloresto.fr/restaurants-livraison-a-domicile/zone-livraison/paris-12"
codeSource = recuperer_page_alloresto(link)
recuperer_liste_restaurants(codeSource, liste_restaurants, liste_plats)

In [10]:
liste_restaurants

[{'address': '25 Rue du Docteur Heulin, 75017 PARIS 17',
  'id': 0,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/dabbawalas-paris75017/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/9849dabwalas200.png',
  'minimum_order': '18.00€',
  'name': 'Dabbawalas',
  'specialties': ['Indien-pakistanais', 'Halal'],
  'work_time': None},
 {'address': '14 Rue de Nice, 75011 PARIS 11',
  'id': 1,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/ete-edo-750110/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/132312.png',
  'minimum_order': '18.00€',
  'name': 'Eté Edo',
  'specialties': ['Japonais', 'Chinois'],
  'work_time': None},
 {'address': '2 - 4 rue de Lagny, 75020 PARIS 20',
  'id': 2,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/sushi-home-75020/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/36103.png'

In [11]:
len(liste_restaurants)

351

In [12]:
liste_plats

[{'descrption': 'Pâte à pain levée fourrée au fromage cuit au tandoor',
  'id': '2162124',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Nan Fromage',
  'prix': '3,50 €'},
 {'descrption': 'Riz nature parfumé',
  'id': '2162131',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Riz Basmati',
  'prix': '3,50 €'},
 {'descrption': 'Galette croustillante avec farine de lentille et graines du cumin',
  'id': '2162127',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Papadum',
  'prix': '0,50 €'},
 {'descrption': 'Nan au choix + Entrée au choix + Plat au choix: Accompagnée du Riz aux choix + Dessert au choix + Boisson ou Bière au choix',
  'id': '2162103',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Menu Dabbawalas',
  'prix': '25,00 €'},
 {'descrption': 'Beignets de viande hachée avec petits pois',
  'id': '2162117',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Samosa Viande',
  'prix': '4,90 €'},
 {'descrption': 'Beignets de légumes avec petits pois',
  'id': '2162118',
  'id_restaurant'

In [13]:
len(liste_plats)

47709

In [15]:
_pickle.dump(liste_restaurants, open('liste_restaurants.txt', 'wb')) 
_pickle.dump(liste_plats, open('liste_plats.txt', 'wb')) 
#obj = cPickle.load(open('save.p', 'rb'))

In [6]:
liste_restaurants = _pickle.load(open('liste_restaurants.txt', 'rb'))
liste_restaurants

[{'address': '25 Rue du Docteur Heulin, 75017 PARIS 17',
  'id': 0,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/dabbawalas-paris75017/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/9849dabwalas200.png',
  'minimum_order': '18.00€',
  'name': 'Dabbawalas',
  'specialties': ['Indien-pakistanais', 'Halal'],
  'work_time': None},
 {'address': '14 Rue de Nice, 75011 PARIS 11',
  'id': 1,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/ete-edo-750110/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/132312.png',
  'minimum_order': '18.00€',
  'name': 'Eté Edo',
  'specialties': ['Japonais', 'Chinois'],
  'work_time': None},
 {'address': '2 - 4 rue de Lagny, 75020 PARIS 20',
  'id': 2,
  'link': 'https://www.alloresto.fr/restaurant-livraison-a-domicile/restaurant/sushi-home-75020/paris-12/particuliers/carte',
  'logo': 'https://www.alloresto.fr/logos_restos/36103.png'

In [7]:
with open('restaurants.json', 'wb')as f:
    f.write(json.dumps(liste_restaurants, ensure_ascii=False, indent = 4).encode('utf8'))

In [3]:
liste_plats = _pickle.load(open('liste_plats.txt', 'rb'))
liste_plats

[{'descrption': 'Pâte à pain levée fourrée au fromage cuit au tandoor',
  'id': '2162124',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Nan Fromage',
  'prix': '3,50 €'},
 {'descrption': 'Riz nature parfumé',
  'id': '2162131',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Riz Basmati',
  'prix': '3,50 €'},
 {'descrption': 'Galette croustillante avec farine de lentille et graines du cumin',
  'id': '2162127',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Papadum',
  'prix': '0,50 €'},
 {'descrption': 'Nan au choix + Entrée au choix + Plat au choix: Accompagnée du Riz aux choix + Dessert au choix + Boisson ou Bière au choix',
  'id': '2162103',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Menu Dabbawalas',
  'prix': '25,00 €'},
 {'descrption': 'Beignets de viande hachée avec petits pois',
  'id': '2162117',
  'id_restaurant': 0,
  'logo': '',
  'name': 'Samosa Viande',
  'prix': '4,90 €'},
 {'descrption': 'Beignets de légumes avec petits pois',
  'id': '2162118',
  'id_restaurant'

In [4]:
with open('plats.json', 'wb')as f:
    f.write(json.dumps(liste_plats, ensure_ascii=False, indent = 4).encode('utf8'))